# OpenFOAM TemporalProbe Analysis

This program shows the evolution of multiple temporal probes for an OpenFOAM case.
    
    @author Daniel Duke <daniel.duke@monash.edu>
    @copyright (c) 2020 LTRAC
    @license GPL-3.0+
    @version 0.0.1
    @date 30/05/2021
        __   ____________    ___    ______
       / /  /_  ____ __  \  /   |  / ____/
      / /    / /   / /_/ / / /| | / /
     / /___ / /   / _, _/ / ___ |/ /_________
    /_____//_/   /_/ |__\/_/  |_|\__________/

    Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)
    Monash University, Australia


In [8]:
# Load modules
import numpy as np
import glob
import tqdm
import matplotlib.pyplot as plt
%matplotlib notebook

In [5]:
case_dir = '/mnt/internal/2021_pmdi/les_pmdimv2_0.33x1.2_trial_134a15pcEtOH_smi'
variables = ['y']

In [ ]:
def analyse_variable(v):
    
    # Get all the data
    files = glob.glob(case_dir+'/temporalProbes/*/'+v)
    
    # Get the time steps
    t0s = [ float(f[len(case_dir)+1:].split('/')[1]) for f in files ]
    
    # Put them in order
    idx = np.argsort(t0s)

    # Read in the data, in order of the starting time of each subset of data.
    data = []
    for i in tqdm.tqdm(idx):
        data.append( np.genfromtxt(files[i]) )
    
    # Arrange the data into a single large matrix, truncating any repeated timesteps at the end of a run
    A = []
    tmax_last = 0.
    for d in data:
        
        t = d[:,0]
        print(d.shape,t.min(),t.max())
        
        if A == []: # copy
            A = d[...] 
        
        elif t.min() >= tmax_last: # no overlap
            A = np.hstack((A, d))
            
        else: # has overlap - discard end of previous run
            deltat = np.abs(A[:,0] - t.min())
            te = np.where( deltat == np.nanmin(deltat) )[0][0]
            A = np.hstack((A[:te], d))
            
        tmax_last = t.max()
        print('\t',A.shape)
        
    del data
    
    fig=plt.figure()
    plt.plot(np.arange(A[:,0]),A[:,0]);
    
    return

for v in variables: analyse_variable(v)

 25%|██▌       | 3/12 [00:02<00:07,  1.25it/s]